# 2. Classification

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import SGD

/home/kai/anaconda3/envs/ml/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/kai/anaconda3/envs/ml/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/kai/anaconda3/envs/ml/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/kai/anaconda3/envs/ml/lib/python3.7/site-packages/tensorflow/

## Classification intro

### Supervised learning

- What is supervised learning?
- Training
- Fitting
- Labels
- Error / cost / objective functions

### Classification

- What is classification?
- Different approaches
  - Logistic regression - log loss
  - ANNs, SVMs
  - ...

### No free lunch

- Model selection.
- Parametric / nonparametric models.
- Discriminative / generative modelling.

### Overfitting

- Overfitting
- Underfitting
- Generalisation

### Optimisation

- Hyperparameters

#### Classification with MNIST

- The aim of classification with MNIST
- Get the data
- Recap analysis
  - Size, shape, type
  - Features
  - Missing data, outliers etc.
  - Target feature of dataset (class)

Download the MNIST dataset, getting features X and labels y:

In [24]:
X, y = fetch_openml('mnist_784', return_X_y=True)

We shall look at the number of unique features in the dataset.

In [25]:
n_classes = len(np.unique(y))
print(n_classes)

10


There appear to be 10 unique digit types.

We shall put the data into a NumPy array and analyse the shape of the data:

In [26]:
X = np.asarray(X)
y = np.asarray(y)

print(f"X shape: {X.shape}, y shape: {y.shape}")
print(f"Image shape: {X[0].shape}")

X shape: (70000, 784), y shape: (70000,)
Image shape: (784,)


From the shape we can see that we have the full MNIST dataset of 70000 hand written didgets. 

They are yet to be split into a training and testing set and each image has been flattened to a (784,) array rather than the standard (28,28) form.

Now we shall have a look at the image data to see what form it is in.

In [27]:
print(np.min(X[0]), np.max(X[0]))

0.0 255.0


Looking at the minimum and maximum values in appears that pixel are of the standard 0-255 intensity form. 

It is beneficial to some machine learning methods for values to be normalised between 0-1, notably neural networks so we shall do this in the data preparation step.

Finally we shall check the datatypes for features and labels.

In [28]:
print(X.dtype)
print(y.dtype)

float64
object


The features are of float type as expected but it may be beneficial to convert the string type labels to integers.

#### Preparing the data

  - train, val test sets
  - cross validation
  - reshape / retype

As stated earlier normalising the feature values will be beneficial for machine learning later on.

In [29]:
X_full = X / 255.0
print(np.min(X_full), np.max(X_full))

0.0 1.0


We shall also now convert the string type features to integers for use later on.

In [30]:
y_full = y.astype('int')
print(y.dtype)

object


We will need to create a train, validation and test set before we proceed using sklearns `train_test_split`.

In [31]:
# train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# remove a validation set from the training data
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [32]:
print(X_full.shape[0])
print(X_train.shape[0])
print(X_val.shape[0])
print(X_test.shape[0])

70000
44800
11200
14000


Now we can train the data and analyse using the validation set before testing on the test set. This will allow us to minimise overfitting by ensuring our model generalises well.

Later on we shall use cross-validation on the entire training set to ensure maximum generalisation, but for now we shall use the the validation set for this.

## 2.1 Artificial Neural Networks (ANNs)

### Neurons

- Neurons structure
- Action potentials ~ activation functions
- Weights, connections etc.

### The perceptron

- Perceptron history
- Input layer
- Output layer
- Activation function
- Weights
- Bias
- Try and apply it to MNIST (lots of neurons?)

In [6]:
# perceptron

### Multi-layer perceptron

- Multiple layers of neurons
- Able to learn more complex patterns

### Training MLPs

- Forward pass
- Backward pass
- Epochs, convergence

### Forward pass

- Calculation of outputs from input
- Calculates loss

### Backpropogation

- Gradient descent
- Learning rate
- Minimise loss function
- Convex optimisation problem
- Chain rule

### Stochastic gradient descent

- What is SGD
- Diagram

### Cross entropy

- What is cross entropy
- Sparse categorical cross entropy

### Classifying MNIST

- Feed forward network
- Dense layers
- Epochs, convergence
- Create a simple MLP for MNIST

In [7]:
model = Sequential()
model.add(Dense(300, activation="relu", input_shape=[784]))
model.add(Dense(100, activation="relu"))
model.add(Dense(10, activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy", optimizer='sgd', metrics=["accuracy"])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [9]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
1400/1400 [==============================] - 4s 3ms/step - loss: 0.6903 - accuracy: 0.8213 - val_loss: 0.3466 - val_accuracy: 0.8991
Epoch 2/10
1400/1400 [==============================] - 3s 2ms/step - loss: 0.3159 - accuracy: 0.9100 - val_loss: 0.2694 - val_accuracy: 0.9198
Epoch 3/10
1400/1400 [==============================] - 3s 2ms/step - loss: 0.2600 - accuracy: 0.9252 - val_loss: 0.2393 - val_accuracy: 0.9301
Epoch 4/10
1400/1400 [==============================] - 4s 3ms/step - loss: 0.2237 - accuracy: 0.9356 - val_loss: 0.2081 - val_accuracy: 0.9381
Epoch 5/10
1400/1400 [==============================] - 3s 2ms/step - loss: 0.1956 - accuracy: 0.9433 - val_loss: 0.1880 - val_accuracy: 0.9433
Epoch 6/10
1400/1400 [==============================] - 3s 2ms/step - loss: 0.1738 - accuracy: 0.9501 - val_loss: 0.1700 - val_accuracy: 0.9488
Epoch 7/10
1400/1400 [==============================] - 3s 2ms/step - loss: 0.1560 - accuracy: 0.9548 - val_loss: 0.1567 - val_accuracy:

### Deeper neural networks

- Create a large overfitting network
- Early stopping, check pointing

In [40]:
model = Sequential()
model.add(Dense(500, activation="relu", input_shape=[784]))
model.add(Dense(400, activation="relu"))
model.add(Dense(300, activation="relu"))
model.add(Dense(200, activation="relu"))
model.add(Dense(100, activation="relu"))
model.add(Dense(50, activation="relu"))
model.add(Dense(25, activation="relu"))
model.add(Dense(10, activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy", optimizer='sgd', metrics=["accuracy"])

In [41]:
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))

Epoch 1/20
1400/1400 [==============================] - 10s 7ms/step - loss: 1.0119 - accuracy: 0.6736 - val_loss: 0.3504 - val_accuracy: 0.8922
Epoch 2/20
1400/1400 [==============================] - 8s 6ms/step - loss: 0.2486 - accuracy: 0.9279 - val_loss: 0.1961 - val_accuracy: 0.9433
Epoch 3/20
1400/1400 [==============================] - 10s 7ms/step - loss: 0.1622 - accuracy: 0.9531 - val_loss: 0.1517 - val_accuracy: 0.9548
Epoch 4/20
1400/1400 [==============================] - 9s 7ms/step - loss: 0.1213 - accuracy: 0.9643 - val_loss: 0.1370 - val_accuracy: 0.9588
Epoch 5/20
1400/1400 [==============================] - 10s 7ms/step - loss: 0.0954 - accuracy: 0.9718 - val_loss: 0.1221 - val_accuracy: 0.9638
Epoch 6/20
1400/1400 [==============================] - 9s 7ms/step - loss: 0.0756 - accuracy: 0.9772 - val_loss: 0.1135 - val_accuracy: 0.9665
Epoch 7/20
1400/1400 [==============================] - 9s 7ms/step - loss: 0.0596 - accuracy: 0.9823 - val_loss: 0.1034 - val_accura

### Vanishing / exploding gradient problem

- What is the vanishing gradient problem

### Initlialisation

- GloRoot initialisation

### Saturating activation functions

- Leaky RELU
- SELU
- Tanh...

### Batching

- Batch normalisation
- Mini-batch
- Implement batching

### Gradient clipping

- Gradient clipping

### Optimisation of ANNs

- Momentum
- Adam
- ...

### Overfitting in ANNs

- Why large networks can overfit
- l1, l2 loss and regularisation
- Implement regularisation
- Dropout, monte carlo dropout
- Create a network with dropout

### Convolutional neural networks (CNN)

- Problems with neural networks and images
- Convolutions, filters
- Convolutional layers, feature maps
- Simple CNN for MNIST

### Training better CNNs

- Stride, step
- Pooling, max pooling
- Better CNN

### Transfer learning

- What is transfer learning
- Use transfer learning with MNIST

### Limitations of ANNs

- Problems with ANNs

## 2.2. Support Vector Machines (SVMs)

Train an SVM (with a chosen Kernel) and perform the same analyses as forANNs. Interpret  and  discuss  your  results. Does the model overfit? How do they compare with ANNs? And why? How does the type of kernel (e.g.linear, RBF, etc.) impact on performance?

### Support vector machines

- Hyperplane
- Support vectors
- Minimising lagrange multiplier (MIT)

### Linear SVM

- Implementation of SVM on MNIST
- Plot decision boundaries from lab 2

### Probabilistic SVM

- What is a probabilistic SVM?
- SVM on MNIST

### The dual problem

What is the dual problem

### Kernels

- Different representations
- Kernels
- Mercer's theorem
- Gram matrix

### The kernel trick

- What is the kernel trick

### Kernel SVMs

- How kernel SVMs work

### Polynomial KSVM

- What is the polynomial kernel
- Implementation

### Radial basis function (RBF) KSVM

- What is the RBF kernel
- Implementation

### Sigmoid KSVM

- What is the sigmoid kernel
- Implementation

### Optimising SVMs

- Hinge loss
- Grid search hyperparameters

### Limitations of SVMs

- Problems with SVMs

## Classification conclusion

#### Comparison of ANNs and SVMs

- Comparison of neural networks and support vector machines.
- Area under ROC curve
- Precision, recall, accuracy...
- TP, TN, FP, FN and rates for each
- Confusion matrix
- Metric
- Validation
- Cross-validation
- Prediction
- Inference
- Interpretability
- Sensitivity vs specificity